<a href="https://colab.research.google.com/github/kavyajeetbora/modern_geospatial_stack/blob/master/use_cases/retail_store_analysis_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Dependencies

In [17]:
## Install required packages
!pip install -q streamlit
!npm install localtunnel
import urllib

## Download required data files
!wget https://github.com/kavyajeetbora/modern_geospatial_stack/raw/master/use_cases/pop_rad_data.parquet -O pop_rad_data.parquet
!wget https://github.com/kavyajeetbora/modern_geospatial_stack/raw/master/use_cases/poi_accessibility.parquet -O poi_accessibility.parquet
!wget https://raw.githubusercontent.com/kavyajeetbora/modern_geospatial_stack/master/use_cases/style.css -O style.css

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.531s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details
--2024-05-07 13:58:13--  https://github.com/kavyajeetbora/modern_geospatial_stack/raw/master/use_cases/pop_rad_data.parquet
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kavyajeetbora/modern_geospatial_stack/master/use_cases/pop_rad_data.parquet [following]
--2024-05-07 13:58:13--  http

## About the Data


### Amenties and POIs

1. Restaurants, offices, malls etc attract foot traffic. A retail store near these can benefit.

2. What data was considered for amenties and POIs?
    - OSM features with tags: `amenity`, `office`, `shop`, `tourism`
    - Here is a comprehensive list of OSM tags: [OSM Features](https://wiki.openstreetmap.org/wiki/Map_features). More features can be added to enhance the results
    - python library called [`osmnx`](https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.features.features_from_polygon) was used for extracting OSM features
    - For visualization data was converted for h3 spatial index and plotted using [`pydeck`](https://deckgl.readthedocs.io/en/latest/gallery/h3_hexagon_layer.html) python library
    - [Analysis Notebook](https://github.com/kavyajeetbora/modern_geospatial_stack/blob/master/use_cases/Location_Intelligence.ipynb)

### Public Transport Accessibility

- To calculate the accessibility to the public transport hubs, `pandana` python library was used
-  Pandana focuses on analyzing networks (e.g., transportation networks, road networks, etc.).
- It calculates metrics related to travel accessibility, such as how easily one can reach different parts of a network
- in this case, OSM features with tag: `public_transport` and `'highway': ['primary', 'secondary', 'tertiary']`
- [Analysis Notebook](https://github.com/kavyajeetbora/modern_geospatial_stack/blob/master/use_cases/Location_Intelligence.ipynb)

### Population Density

- Data used to estimate the population density spaitally: [WorldPop Global Project Population Data](https://developers.google.com/earth-engine/datasets/catalog/WorldPop_GP_100m_pop)
- Estimates the population count per 100m x 100m grid
- WorldPop combines census data, geospatial information, and modeling techniques to estimate population counts at various spatial scales. These estimates are valuable for decision-makers, researchers, and humanitarian efforts worldwide
- [How population is estimated ?](https://www.worldpop.org/methods/populations/)

### Radiance Values

- Dataset used: [VIIRS Nighttime Data](https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG)

    <img src="https://developers.google.com/earth-engine/datasets/images/NOAA/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG_sample.png" height=200/>

- Understanding VIIRS DNB Data:

    - The VIIRS DNB captures radiance values in the visible and near-infrared spectrum during both day and night.
    - These radiance values represent the amount of light detected by the sensor at nighttime.
    - Higher radiance values indicate brighter sources of light (e.g., urban areas, artificial lighting).
    - Lower radiance values correspond to darker regions with less light
    - Resolution = 463.83 meters, radiance values are in Nano Watts per area



## Streamlit Application

In [19]:
%%writefile app.py

import streamlit as st
import pydeck as pdk
import pandas as pd
from matplotlib import colormaps
from sklearn.preprocessing import MinMaxScaler
import matplotlib as mpl
import matplotlib.pyplot as plt

COLOR = 'blue'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

def return_legend(series, cmap, units):
    fig, ax = plt.subplots(figsize=(5, 1), layout='constrained')
    norm = mpl.colors.Normalize(vmin=series.min(), vmax=series.max())
    fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                cax=ax, orientation='horizontal', label=units)

    return fig

## STREAMLIT CONFIGURATION
## --------------------------------------------------------------------------------##
st.set_page_config(page_title='Retail Store Location Analysis', page_icon='', layout="wide")


## Streamlit CSS styling
## --------------------------------------------------------------------------------##
margins_css = """
    <style>
        #root > div:nth-child(1) > div.withScreencast > div > div > div > section.main.st-emotion-cache-bm2z3a.ea3mdgi8 > div.block-container.st-emotion-cache-1jicfl2.ea3mdgi5 {
            padding-left: 0rem;
            padding-right: 20px;
            padding-top: 20px;
        }

        #root > div:nth-child(1) > div.withScreencast > div > div > div > section.main.st-emotion-cache-bm2z3a.ea3mdgi8 > div.block-container.st-emotion-cache-1jicfl2.ea3mdgi5 > div > div > div > div:nth-child(4) > div > div {
            height:1000px;
        }

        .st-emotion-cache-1v0mbdj {
            width: 25rem;
        }
    </style>
"""
st.markdown(margins_css, unsafe_allow_html=True)

## Read the data files
## --------------------------------------------------------------------------------##

pop_rad_data = pd.read_parquet('pop_rad_data.parquet').fillna(0)
pop_rad_data['radiance'] = pop_rad_data['radiance'].astype(int)
poi_accessibility = pd.read_parquet('poi_accessibility.parquet').fillna(0)

## Create the overall suitability score
## --------------------------------------------------------------------------------##
scaler = MinMaxScaler()
df = pop_rad_data[['hex8','radiance', 'population']].set_index('hex8').join(poi_accessibility[['hex8','poi_count', 'time_min']].set_index('hex8'))
df[['radiance', 'population','poi_count', 'time_min']] = scaler.fit_transform(df[['radiance', 'population','poi_count', 'time_min']])
df['time_min'] = 1-df['time_min']
df['suitablity_score'] = df.sum(axis=1)
df = df[['suitablity_score']].reset_index()
scaler = MinMaxScaler()
df['suitablity_score'] = (scaler.fit_transform(df['suitablity_score'].values.reshape(-1,1)).ravel()*100).astype(int)


## UTILITY Functions
## --------------------------------------------------------------------------------##

def color_code_value(value, cmap):
    color = cmap(value)
    scaled_colors = list(map(lambda x: int(x * 255), color[:3]))
    scaled_alpha = int(color[3] * 100)

    scaled_colors += [scaled_alpha]
    return scaled_colors


def colormap_dataframe(df, value_col, cmap):

    xdf = df.copy()

    ## Scale the temperature values between 0-1; using MinMax Scaler
    xdf[f"norm_{value_col}"] = (xdf[value_col] - xdf[value_col].min()) / (
        xdf[value_col].max() - xdf[value_col].min()
    )
    xdf["color"] = xdf[f"norm_{value_col}"].apply(lambda x: color_code_value(x, cmap))
    xdf[["R", "G", "B", "A"]] = pd.DataFrame(xdf["color"].to_list())
    xdf = xdf.drop(["color", f"norm_{value_col}"], axis=1)

    ## formatting the temperature value upto 1 decimal place
    xdf[value_col] = xdf[value_col].round(1)

    return xdf


def create_h3_hex_layer(df, hex_col):

    # Define a layer to display on a map
    layer = pdk.Layer(
        "H3HexagonLayer",
        df,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        get_hexagon=hex_col,
        get_fill_color="[R, G, B, A]",
    )

    return layer

## Generate Hex Layer for each analysis
## --------------------------------------------------------------------------------##
hex_col = 'hex8'

## Radiance data
cmap=colormaps['cividis']
radiance = colormap_dataframe(pop_rad_data[['hex8', 'radiance']], value_col='radiance', cmap=cmap)
rad_layer = create_h3_hex_layer(radiance, hex_col)
rad_legend = return_legend(radiance['radiance'],cmap, units="Radiance")

# Population data
cmap=colormaps['seismic']
pop_age_sex = colormap_dataframe(pop_rad_data[['hex8', 'population', 'sum_male', 'sum_female']], value_col='population', cmap=cmap)
pop_age_sex_layer = create_h3_hex_layer(pop_age_sex, hex_col)
pop_age_sex_legend = return_legend(pop_age_sex['population'],cmap, units="Population Count")

# Density of Amenity
cmap = colormaps['viridis']
poi_density = colormap_dataframe(poi_accessibility[['hex8', 'poi_count']], value_col='poi_count', cmap=cmap)
poi_density_layer = create_h3_hex_layer(poi_density, hex_col)
poi_density_legend = return_legend(poi_density['poi_count'],cmap, units='Density of Amenity')

# Accessibility
cmap = colormaps['YlOrRd']
accessibility = colormap_dataframe(poi_accessibility[['hex8', 'time_min']], value_col='time_min', cmap=cmap)
accessibility_layer = create_h3_hex_layer(accessibility, hex_col)
accessibility_legend = return_legend(accessibility['time_min'],cmap, units='Public Transport Accessibility')

## suitability score
score = colormap_dataframe(df[['hex8', 'suitablity_score']], value_col='suitablity_score', cmap=colormaps['YlOrRd'])
score_layer = create_h3_hex_layer(score, hex_col)
score_legend = return_legend(score['suitablity_score'],cmap, units='Retail Location Suitability Score')

def generate_tooltip(text):
    tooltip = {
            "html": text,
            "style": {
                "backgroundColor": "#4CAF50",   # Green shade for background
                "color": "#FFFFFF",             # White for text color
                "border": "2px solid #4CAF50",  # Matching border color
                "borderRadius": "5px",          # Rounded corners
                "boxShadow": "2px 2px 10px rgba(0, 0, 0, 0.2)"  # Soft shadow effect
            }
        }

    return tooltip

### Create the description for each analysis

poi_desc = '''
### Amenties and POIs

1. Restaurants, offices, malls etc attract foot traffic. A retail store near these can benefit.

2. What data was considered for amenties and POIs?
    - OSM features with tags: `amenity`, `office`, `shop`, `tourism`
    - Here is a comprehensive list of OSM tags: [OSM Features](https://wiki.openstreetmap.org/wiki/Map_features). More features can be added to enhance the results
    - python library called [`osmnx`](https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.features.features_from_polygon) was used for extracting OSM features
    - For visualization data was converted for h3 spatial index and plotted using [`pydeck`](https://deckgl.readthedocs.io/en/latest/gallery/h3_hexagon_layer.html) python library
    - [Analysis Notebook](https://github.com/kavyajeetbora/modern_geospatial_stack/blob/master/use_cases/Location_Intelligence.ipynb)
'''

## Generate the dropdown

options = {
    "Density of Amenities": (poi_density_layer, generate_tooltip("Amenities Count: {poi_count}"), poi_density_legend),
    "Public Transport Accessibility": (accessibility_layer, generate_tooltip("Pedestrian Accessibility: {time_min} mins"), accessibility_legend),
    "Population Density": (pop_age_sex_layer, generate_tooltip('''Population: {population} <br>
        Male: {sum_male} % <br>
        Female: {sum_female} %
        '''), pop_age_sex_legend),
    "Nighttime Lights": (rad_layer, generate_tooltip("Radiance: {radiance}"),rad_legend),
    "Suitability Score": (score_layer, generate_tooltip("Suitability Score: {suitablity_score} %"),score_legend)

}

def create_deck(hex_layer, tooltip):
    x, y = (77.06463665845695, 28.437398083125167)
    view_state = pdk.ViewState(latitude=y, longitude=x, zoom=12, bearing=0, pitch=45)
    # Render
    r = pdk.Deck(layers=[hex_layer], initial_view_state=view_state, tooltip=tooltip)

    return r

## Frond-end of the application
## --------------------------------------------------------------------------------##

st.sidebar.markdown("## Map Controls")

parameter = st.sidebar.selectbox(
    label="Parameter",
    options=options.keys(),
)

hex_layer, tooltip, legend = options[parameter]

st.header(parameter)
deck = create_deck(hex_layer, tooltip)

col1, col2 = st.columns(2)

with col1:
    st.pydeck_chart(deck, use_container_width=True)
    st.pyplot(legend,use_container_width=False)

with col2:


Overwriting app.py


## Deploy the application

In [ ]:
password = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")
print("Password/Enpoint IP for localtunnel is:", password)

!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501

Password/Enpoint IP for localtunnel is: 35.194.155.84
npx: installed 22 in 2.329s
your url is: https://crazy-points-warn.loca.lt


## TODO

- ~Add the ranking summary plot~
- ~Add legend for each plot~
- Try full screen for the plot